## Importing the required modules

In [ ]:
from urllib.request import urljoin
from bs4 import BeautifulSoup
import requests
from urllib.request import urlparse
import re
from lxml import etree

In [ ]:
input_url = "https://www.boxofficemojo.com/year/"
no_of_years = 6
no_of_movies = 3

In [ ]:
current_url_domain = urlparse(input_url).netloc
print(current_url_domain)

www.boxofficemojo.com


## Extracting page content of Input URL 

In [ ]:
# The function returns a beautiful soup object containing the entire HTML content
beautiful_soup_object = BeautifulSoup(requests.get(input_url).content, "lxml")

In [ ]:
queue = []
links = beautiful_soup_object.findAll("a")    # Since we only require Links to crawl, finding all anchor tags(a)

In [ ]:
links

In [ ]:
# From all links obtained, we only need the links of Years
regex_expression = '/year/\d\d\d\d/.*'
check = re.compile(regex_expression)
count = 0

for anchor in links:
    href = anchor.attrs.get("href")
    s = str(href)
    mo = check.search(s)              # takes matching objects   
    
    # Extracting years links from page1
    if mo != None and count < no_of_years:
        # print(mo.group(0))
        queue.append(mo.group(0))
        count = count + 1

In [ ]:
length_of_queue_at_end_of_depth0 = len(queue)
print(queue)

['/year/2021/?ref_=bo_yl_table_1', '/year/2020/?ref_=bo_yl_table_2', '/year/2019/?ref_=bo_yl_table_3', '/year/2018/?ref_=bo_yl_table_4', '/year/2017/?ref_=bo_yl_table_5', '/year/2016/?ref_=bo_yl_table_6']


In [ ]:
#depth 1
i = 0
while i < length_of_queue_at_end_of_depth0:
    year = queue.pop(0)
    year_link = urljoin(input_url, year)
    
    beautiful_soup_object = BeautifulSoup(requests.get(year_link).content, "lxml")
    
    regex_expression1 = '/release/.*' 
    check1 = re.compile(regex_expression1)
    count = 0
    
    for anchor in beautiful_soup_object.findAll("a"):
        href = anchor.attrs.get("href")
        s = str(href)
        mo = check1.search(s)
        if mo != None and count < no_of_movies:
            queue.append(mo.group(0))
            count = count + 1
        
        elif count == no_of_movies:
            break
    
    i = i + 1

In [ ]:
print(len(queue))

18


In [ ]:
input_url

'https://www.boxofficemojo.com/year/'

In [ ]:
#depth2
movie_links = []
while len(queue) != 0:
    movie = queue.pop(0)
    # print(movie)
    movie_links.append(urljoin(input_url, movie))
    # print(movie_links)

In [ ]:
# Helper functions that capture and return the required regex groups
def getDetails(movieDetails, regexp):
  pattern = re.compile(regexp)
  match = pattern.search(str(movieDetails))
  if(match):
    # print(match.group(1))
    return match.group(1)
  return None


def getReleaseDate(movieDetails, regexp):
  pattern = re.compile(regexp)
  match = pattern.search(str(movieDetails))
  if(match):
    # print(match.group(2))
    return match.group(2)
  return None

In [ ]:
def crawlMovies(pageContents, currentPage):
    titleObj = pageContents.findAll("h1", {"class" : "a-size-extra-large"})
    movieSummaryObj = pageContents.findAll("p", {"class" : "a-size-medium"})
    grossObj = pageContents.find(class_ = "a-section a-spacing-none mojo-performance-summary-table").find_all(class_ = "money")
    

    # print(titleObj)
    # converting the page
    # print("Current Page Link " + currentPage)
    
    detailsObj = BeautifulSoup(requests.get(currentPage).content, "lxml")
    dom = etree.HTML(str(detailsObj))

    # Extracting inner text from title object
    title = ""
    for movieTitle in titleObj:
      title += movieTitle.text

    # print("Title   :" + title)

    # Extracting inner text from movieSummary object
    movieSummary = ""
    for summary in movieSummaryObj:
      movieSummary += summary.text

    # print("movieSummary   :" + movieSummary)

    # Extracting the world wide gross for the movie
    worldWideGross = "";
    for gross in grossObj:
      worldWideGross = gross.text

    movieDetails = pageContents.findAll("div", {"class":"a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile"})
    # print(movieDetails)

    distributor = ""
    distributorRegex = '<div class="a-section a-spacing-none"><span>Distributor<\/span><span>([a-zA-Z\s()]+)'
    distributor = getDetails(movieDetails, distributorRegex)
    # print(distributor)

    budget = "";
    budgetRegex = '<div class="a-section a-spacing-none"><span>Budget<\/span><span><span class="money">([a-zA-Z\d$,]+)'
    budget = getDetails(movieDetails, budgetRegex)
    # print(budget)

    releaseDate = "";
    # releaseDateRegex = 'div class="a-section a-spacing-none"><span>Release Date<\/span><span><a class="a-link-normal" href="\/date\/[\d-]+\/\?ref_=bo_rl_rl">([a-zA-Z\s\d,]+)'
    releaseDateRegex = '<div class="a-section a-spacing-none"><span>Release Date([a-zA-Z\s()]+)?<\/span><span><a class="a-link-normal" href="\/date\/[\d-]+\/\?ref_=bo_rl_rl">([a-zA-Z\s\d,]+)'
    releaseDate = getReleaseDate(movieDetails, releaseDateRegex)
    # print(releaseDate)

    genre = ""
    genreRegex = '<div class="a-section a-spacing-none"><span>Genres<\/span><span>([a-zA-Z\s-]+)'
    genre = (getDetails(movieDetails, genreRegex)).strip()
    
    # Since genre is stored as newlines in page content, using regex to remove unwanted \n and spaces
    genre = re.sub('\\n\\s+', ",", genre)
    # print(genre)

    runningTime = ""
    runTimeRegex = '<div class="a-section a-spacing-none"><span>Running Time<\/span><span>([a-zA-Z0-9\s]+)'
    runningTime = getDetails(movieDetails, runTimeRegex)
    # print(runningTime)


    # Printing details of movies
    print('Movie Name        : ' + title)
    print('Movie Summary     : ' + movieSummary)
    print('Distributor       : ' + distributor)
    print('Release Date      : ' + releaseDate)
    if(budget):
      print('Movie Budget      : ' + budget)
    print('Running Time      : ' + runningTime)
    print('Genre             : ' + genre)
    print('Worldwide Gross   : ' + worldWideGross)

In [ ]:
for currentPage in movie_links:
  pageContents = BeautifulSoup(requests.get(currentPage).content, "lxml")
  crawlMovies(pageContents, currentPage)
  print("\n")
  print('-'*200)
  print("\n")

[<h1 class="a-size-extra-large">Shang-Chi and the Legend of the Ten Rings</h1>]
Movie Name        : Shang-Chi and the Legend of the Ten Rings
Movie Summary     : Shang-Chi, the master of weaponry-based Kung Fu, is forced to confront his past after being drawn into the Ten Rings organization.
Distributor       : Walt Disney Studios Motion Pictures
Release Date      : Sep 3, 2021
Running Time      : 2 hr 12 min
Genre             : Action,Adventure,Fantasy,Sci-Fi
Worldwide Gross   : $430,523,828


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


[<h1 class="a-size-extra-large">Venom: Let There Be Carnage</h1>]
Movie Name        : Venom: Let There Be Carnage
Movie Summary     : Eddie Brock attempts to reignite his career by interviewing serial killer Cletus Kasady, who becomes the host of the symbiote Carnage and escapes prison after a fa